In [1]:
import tensorflow as tf
import numpy as np

# preprocessed data
from datasets.danijela import data
import data_utils
import s2s

In [2]:

# load data from pickle and npy files
metadata, idx_q, idx_a = data.load_data(PATH='datasets/danijela/')
(trainX, trainY), (testX, testY), (validX, validY) = data_utils.split_dataset(idx_q, idx_a)

# parameters 
x_len = trainX.shape[-1]
y_len = trainY.shape[-1]
batch_size = 64
x_vocab_size = len(metadata['idx2w'])  
y_vocab_size = x_vocab_size
emb_dim = 364
num_layers = 2

model = s2s.S2S(x_len=x_len,
                y_len=y_len,
                x_vocab_size=x_vocab_size,
                y_vocab_size=y_vocab_size,
                ckpt_path='ckpt/dan/',
                emb_dim=emb_dim,
                num_layers=num_layers)

Building graph...
Done...


In [3]:
sess = model.restore_last_session()

In [103]:
test_batch_gen = data_utils.rand_batch_gen(testX, testY, 10)

input_ = test_batch_gen.__next__()[0]

print(input_.shape)
#print(input_)
output = model.predict(sess, input_)

replies = []
for ii, oi in zip(input_.T, output):
    q = data_utils.decode(sequence=ii, lookup=metadata['idx2w'], separator=' ')
    decoded = data_utils.decode(sequence=oi, lookup=metadata['idx2w'], separator=' ').split(' ')
    
    #print(ii)
    #print(q)
    #print(decoded)

    if decoded.count('unk') == 0:
        if decoded not in replies:
            print('q: [{0}]; a: [{1}]'.format(q, ' '.join(decoded)))
            replies.append(decoded)

(25, 10)
q: [poskupljuje pivo]; a: [uci :p e sta ti]
q: [ma to :d evo smor mi, hteo da idemo na caj odo sitra kuci]; a: [nadoknadi ti za 5 za koga mi budem ljuta]
q: [ahaa odo <3]; a: [aj sta ce sa mnom]
q: [ucite li? :d]; a: [jok  ako pisem]
q: [kazna od kontrole komentar na onu fotku sa tvrdjave]; a: [nezz zavisi kad mojo zavrse ovde]
q: [good.]; a: [ma sta ti  smajli]
q: [kako je na malinjaku? :p]; a: [pa dejane majku mu jeel uvek moras biti najgori :d]
q: [69.5/70 pojeo sam ovaj fakultet e]; a: [onda te briga :d pa majmune]
q: [e na unk sam jos]; a: [jebo tebew jaoo ala si bosanac nista cekaj cimera]
q: [a sta sa njim samo? :p]; a: [idii mrs a nije jos sto sam snimala da sam polozila ovi reci bi reci :d]


In [6]:
def sentence_to_indexes(sen):
    words = sen.split(' ')
    
    indices = []
    
    for word in words:
        indices.append(metadata['w2idx'][word])
    
    indices = indices + [0]*(25 - len(indices))
    
    arr = np.array([indices])
    arr = np.transpose(arr)
    return arr

def ask(str):
    input_ = sentence_to_indexes(str)

    #print(input_.shape)
    #print(input_)
    output = model.predict(sess, input_)
    #print(output)
    #test = model.advance_predict(sess, input_)

    for i in output:
        decoded = data_utils.decode(sequence=i, lookup=metadata['idx2w'], separator=' ').split(' ')
        print('q: [{0}]; a: [{1}]'.format(str, ' '.join(decoded)))
        #print(' '.join(decoded))


In [101]:
ask('')

q: [evo ne znam sta da radim sve mi dosadilo vise :d]; a: [milinaaa sad se lep stalno sto]


In [82]:
q: [sta radis]; a: [nistaa smor brate]
q: [cao]; a: [e cao :d]
q: [kravo jedna]; a: [volim i ja tebe :p ajmo na pivo :d]
q: [kad ces u ns ?]; a: [dok ne ustanem ne znam mozda pila 12]
q: [sta delas]; a: [ma smaram se skoro sam zaspala ali se ne odjavljujem :d sta ti?]
q: [kad ces u teslic]; a: [posle juna :d]
q: [?]; a: [sta ti nije jasno]
q: [. . .]; a: [sta jeeeeee sta mee seen razbicu tee docu sad ispred zgrade i zvonicu ti na interfon do ujutru]

(4,)
